In [1]:
import numpy as np
from feature_selection import (construct_mutual_info_relevance_vector, 
                               construct_pearson_corr_relevance_vector,
                               construct_mutual_info_redundancy_matrix,
                               construct_pearson_corr_redundancy_matrix,
                               quadratic_programming_feature_selection,
                               greedy_mrmr_feature_selection,
                               generate_reduced_quadratic_program_with_qpfs)

In [2]:
## Create the random instance
num_data_points = 100
radius = 1.0
d = 3

num_chosen_features = d

# feature vector i: (x_i, y_i, z_i, sin(x_i), y_i**2)
# two balls: (0, 0, 0, radius=1.0), (10.0, 10.0, 10.0, r=1.0)
centers = np.array([[0.0]*d, [10.0]*d])
noise = np.random.normal(0.0, radius, size=(num_data_points, d))
X = np.vstack([noise + centers[0], noise + centers[1]])
sin_x = np.expand_dims(np.sin(X[:, 0]), axis=1)
y2 = np.expand_dims(X[:, 1]**2, axis=1)
yz = np.expand_dims(X[:, 1]*X[:, 2], axis=1)
w = np.random.uniform(0.0, 1.0, size=(num_data_points*2, 1))
X = np.hstack([X, y2, yz, sin_x, w])

column_std = np.std(X, axis=0)
column_mean = np.mean(X, axis=0)
X = (X - column_mean[np.newaxis, :]) / column_std[np.newaxis, :]

X = np.round(X, decimals=2)
y = np.array([0]*num_data_points + [1]*num_data_points)

print(X)
print(y)

[[-0.84 -1.05 -1.28 ... -0.97  1.28  1.6 ]
 [-1.19 -1.19 -1.1  ... -0.97 -1.07  1.29]
 [-0.95 -1.03 -1.07 ... -0.98  0.48 -1.33]
 ...
 [ 1.04  1.13  0.75 ...  0.88 -0.86  1.11]
 [ 1.2   1.02  1.03 ...  1.08 -1.27  0.12]
 [ 1.19  0.81  0.52 ...  0.39 -1.28 -0.06]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [3]:
# X = np.array([[-0.8,  -1.04,  0.53, -0.72],
#             [ 1.13, -0.69, -1.06,  0.91],
#             [ 2.01,  0.41, -0.53,  0.9 ],
#             [-0.28,  0.04, -0.71, -0.28],
#             [-0.86, -0.87, -0.39, -0.76],
#             [ 1.1,   0.6, -0.03,  0.89],
#             [ 0.78, -0.62,  0.92,  0.71],
#             [-1.07, -0.53,  1.39, -0.88],
#             [ 0.08,  0.37,  0.16,  0.08],
#             [-0.95, -1.27, -0.05, -0.81],
#             [ 9.2,   8.96, 10.53,  0.22],
#             [11.13,  9.31,  8.94, -0.99],
#             [12.01, 10.41,  9.47, -0.53],
#             [ 9.72, 10.04,  9.29, -0.29],
#             [ 9.14,  9.13,  9.61,  0.28],
#             [11.1,  10.6,   9.97, -0.99],
#             [10.78,  9.38, 10.92, -0.98],
#             [ 8.93,  9.47, 11.39,  0.48],
#             [10.08, 10.37, 10.16, -0.61],
#             [ 9.05,  8.73,  9.95,  0.37]])

# y = np.array([0]*10+[1]*10).astype(np.int32)

In [4]:
f_pc = construct_pearson_corr_relevance_vector(X, y)
Q_pc = construct_pearson_corr_redundancy_matrix(X)
print("Relevance vector based on Pearson correlation: \n {}".format(f_pc))
print("Redundancy matrix based on Pearson correlation: \n {}".format(Q_pc))

Relevance vector based on Pearson correlation: 
 [0.98397962 0.98204604 0.97402733 0.96463016 0.97754247 0.35684535
 0.03485019]
Redundancy matrix based on Pearson correlation: 
 [[1.         0.97166238 0.9573749  0.95485702 0.9646088  0.35049551
  0.03788575]
 [0.97166238 1.         0.95677215 0.98265617 0.97789392 0.35228556
  0.03213269]
 [0.9573749  0.95677215 1.         0.93976027 0.97863613 0.34268924
  0.01859781]
 [0.95485702 0.98265617 0.93976027 1.         0.97796805 0.38160228
  0.03136601]
 [0.9646088  0.97789392 0.97863613 0.97796805 1.         0.36410594
  0.02148089]
 [0.35049551 0.35228556 0.34268924 0.38160228 0.36410594 1.
  0.05326469]
 [0.03788575 0.03213269 0.01859781 0.03136601 0.02148089 0.05326469
  1.        ]]


In [5]:
bar_q_pc = np.mean(Q_pc)
bar_f_pc = np.mean(f_pc)
alpha_pc = bar_q_pc / (bar_q_pc + bar_f_pc)
print(alpha_pc)

0.4507447019300367


In [6]:
f_mi = construct_mutual_info_relevance_vector(X, y, seed=42)
Q_mi = construct_mutual_info_redundancy_matrix(X, seed=42)
print("Relevance vector based on mutual information: \n {}".format(f_mi))
print("Redundancy matrix based on mutual information: \n {}".format(Q_mi))

Relevance vector based on mutual information: 
 [0.69565343 0.69565343 0.69565343 0.69565343 0.69565343 0.17336299
 0.00275323]
Redundancy matrix based on mutual information: 
 [[3.76901904 0.82621343 0.65184539 0.74846047 0.68384493 1.96771013
  0.        ]
 [0.82621343 3.80213214 0.64765744 2.53554806 0.871391   0.17636161
  0.        ]
 [0.65184539 0.64765744 3.81935833 0.69983857 0.90040483 0.0968675
  0.01926582]
 [0.74846047 2.53554806 0.69983857 3.04299602 0.95734372 0.19043753
  0.02444199]
 [0.68384493 0.871391   0.90040483 0.95734372 3.24765007 0.13269984
  0.08043704]
 [1.96771013 0.17636161 0.0968675  0.19043753 0.13269984 3.88488809
  0.        ]
 [0.         0.         0.01926582 0.02444199 0.08043704 0.
  3.93661428]]


In [7]:
bar_q_mi = np.mean(Q_mi)
bar_f_mi = np.mean(f_mi)
alpha_mi = bar_q_mi / (bar_q_mi + bar_f_mi)
print(alpha_mi)

0.6612048976950432


In [8]:
chosen_ones, feature_weights = quadratic_programming_feature_selection(Q_pc, f_pc, num_chosen_features, alpha=2.0*alpha_pc)
print("Results of QPFS based on Pearson correlation:")
print("Chosen features: {}".format(chosen_ones))
print("Feature weights = {}".format(feature_weights))

     pcost       dcost       gap    pres   dres
 0: -1.1707e+00 -1.8578e+00  1e+01  4e+00  2e+00
 1: -7.5576e-01 -1.6365e+00  9e-01  7e-16  3e-16
 2: -8.0363e-01 -8.4750e-01  4e-02  6e-17  2e-16
 3: -8.3460e-01 -8.3863e-01  4e-03  1e-16  2e-16
 4: -8.3761e-01 -8.3806e-01  4e-04  1e-16  2e-16
 5: -8.3789e-01 -8.3789e-01  8e-06  1e-16  1e-16
 6: -8.3789e-01 -8.3789e-01  8e-08  2e-16  1e-16
Optimal solution found.
Results of QPFS based on Pearson correlation:
Chosen features: [0 1 4]
Feature weights = [8.12196996e-01 1.87792550e-01 2.32167044e-06 4.66760010e-07
 7.64789017e-06 1.08493466e-08 7.14563329e-09]


In [9]:
chosen_ones, feature_weights = quadratic_programming_feature_selection(Q_mi, f_mi, num_chosen_features, alpha=alpha_mi)
print("Results of QPFS based on mutual information:")
print("Chosen features: {}".format(chosen_ones))
print("Feature weights = {}".format(feature_weights))

     pcost       dcost       gap    pres   dres
 0: -2.0878e-01 -1.2429e+00  1e+00  2e-16  3e+00
 1: -2.1131e-01 -2.4517e-01  3e-02  4e-17  1e-01
 2: -2.1695e-01 -2.2031e-01  3e-03  3e-17  5e-03
 3: -2.1807e-01 -2.1829e-01  2e-04  7e-17  1e-16
 4: -2.1814e-01 -2.1815e-01  1e-05  9e-17  8e-17
 5: -2.1814e-01 -2.1814e-01  2e-07  5e-17  1e-16
 6: -2.1814e-01 -2.1814e-01  2e-09  7e-17  9e-17
Optimal solution found.
Results of QPFS based on mutual information:
Chosen features: [4 0 2]
Feature weights = [2.32883822e-01 9.29936437e-02 2.21074042e-01 2.03936799e-01
 2.40278148e-01 1.00636821e-09 8.83354486e-03]


In [10]:
# chosen_ones, mrmr, avg_relevance, avg_redundancy = greedy_mrmr_feature_selection(Q_pc, f_pc, num_chosen_features, seed=42)
# print("Results of MRMR based on Pearson correlation:")
# print("Chosen features: {}".format(chosen_ones))
# print("mRMR = {}".format(mrmr))
# print("Average relevance = {}".format(avg_relevance))
# print("Average redundancy = {}".format(avg_redundancy))

In [11]:
# chosen_ones, mrmr, avg_relevance, avg_redundancy = greedy_mrmr_feature_selection(Q_mi, f_mi, num_chosen_features, seed=42)
# print("Results of MRMR based on mutual information:")
# print("Chosen features: {}".format(chosen_ones))
# print("mRMR = {}".format(mrmr))
# print("Average relevance = {}".format(avg_relevance))
# print("Average redundancy = {}".format(avg_redundancy))

In [12]:
num_active_features = 3
num_frozen_features = 2

In [13]:
generate_reduced_quadratic_program_with_qpfs(Q_pc, f_pc, num_active_features, 
                                             num_frozen_features=num_frozen_features, 
                                             alpha = 2 * alpha_pc,                                             
                                             frozen_vector_strategy="QUBO")

     pcost       dcost       gap    pres   dres
 0: -1.1707e+00 -1.8578e+00  1e+01  4e+00  2e+00
 1: -7.5576e-01 -1.6365e+00  9e-01  7e-16  3e-16
 2: -8.0363e-01 -8.4750e-01  4e-02  6e-17  2e-16
 3: -8.3460e-01 -8.3863e-01  4e-03  1e-16  2e-16
 4: -8.3761e-01 -8.3806e-01  4e-04  1e-16  2e-16
 5: -8.3789e-01 -8.3789e-01  8e-06  1e-16  1e-16
 6: -8.3789e-01 -8.3789e-01  8e-08  2e-16  1e-16
Optimal solution found.


(array([[0.0985106 , 0.09257634, 0.09634022],
        [0.09257634, 0.0985106 , 0.09640603],
        [0.09634022, 0.09640603, 0.0985106 ]]),
 array([-0.67873829, -0.68951155, -0.68988708]),
 -1.578121660728338,
 array([6, 5]),
 array([3, 2, 4]),
 array([1, 0]))

In [14]:
generate_reduced_quadratic_program_with_qpfs(Q_pc, f_pc, num_active_features, 
                                             num_frozen_features=num_frozen_features, 
                                             alpha = 2 * alpha_pc,
                                             frozen_vector_strategy="QPFS")

     pcost       dcost       gap    pres   dres
 0: -1.1707e+00 -1.8578e+00  1e+01  4e+00  2e+00
 1: -7.5576e-01 -1.6365e+00  9e-01  7e-16  3e-16
 2: -8.0363e-01 -8.4750e-01  4e-02  6e-17  2e-16
 3: -8.3460e-01 -8.3863e-01  4e-03  1e-16  2e-16
 4: -8.3761e-01 -8.3806e-01  4e-04  1e-16  2e-16
 5: -8.3789e-01 -8.3789e-01  8e-06  1e-16  1e-16
 6: -8.3789e-01 -8.3789e-01  8e-08  2e-16  1e-16
Optimal solution found.


(array([[0.0985106 , 0.09257634, 0.09634022],
        [0.09257634, 0.0985106 , 0.09640603],
        [0.09634022, 0.09640603, 0.0985106 ]]),
 array([-0.77502705, -0.78377588, -0.78597522]),
 -0.837882096985241,
 array([6, 5]),
 array([3, 2, 4]),
 array([1, 0]))

In [15]:
generate_reduced_quadratic_program_with_qpfs(Q_pc, f_pc, num_active_features, 
                                             num_frozen_features=num_frozen_features, 
                                             alpha = 2 * alpha_pc,                                             
                                             frozen_vector_strategy="hybrid")

     pcost       dcost       gap    pres   dres
 0: -1.1707e+00 -1.8578e+00  1e+01  4e+00  2e+00
 1: -7.5576e-01 -1.6365e+00  9e-01  7e-16  3e-16
 2: -8.0363e-01 -8.4750e-01  4e-02  6e-17  2e-16
 3: -8.3460e-01 -8.3863e-01  4e-03  1e-16  2e-16
 4: -8.3761e-01 -8.3806e-01  4e-04  1e-16  2e-16
 5: -8.3789e-01 -8.3789e-01  8e-06  1e-16  1e-16
 6: -8.3789e-01 -8.3789e-01  8e-08  2e-16  1e-16
Optimal solution found.


(array([[0.0985106 , 0.09257634, 0.09634022],
        [0.09257634, 0.0985106 , 0.09640603],
        [0.09634022, 0.09640603, 0.0985106 ]]),
 array([-0.77417208, -0.78379442, -0.78556663]),
 -0.8376099903326558,
 array([6, 5]),
 array([3, 2, 4]),
 array([1, 0]))

In [16]:
generate_reduced_quadratic_program_with_qpfs(Q_mi, f_mi, num_active_features, 
                                             num_frozen_features=num_frozen_features, 
                                             alpha = alpha_mi,
                                             frozen_vector_strategy="QUBO")

     pcost       dcost       gap    pres   dres
 0: -2.0878e-01 -1.2429e+00  1e+00  2e-16  3e+00
 1: -2.1131e-01 -2.4517e-01  3e-02  4e-17  1e-01
 2: -2.1695e-01 -2.2031e-01  3e-03  3e-17  5e-03
 3: -2.1807e-01 -2.1829e-01  2e-04  7e-17  1e-16
 4: -2.1814e-01 -2.1815e-01  1e-05  9e-17  8e-17
 5: -2.1814e-01 -2.1814e-01  2e-07  5e-17  1e-16
 6: -2.1814e-01 -2.1814e-01  2e-09  7e-17  9e-17
Optimal solution found.


(array([[1.28814375, 0.85903127, 0.21942317],
        [0.85903127, 1.03095215, 0.23710188],
        [0.21942317, 0.23710188, 1.2939799 ]]),
 array([0.11517061, 0.11794865, 0.06592532]),
 0.5003509682609599,
 array([5, 6]),
 array([1, 3, 2]),
 array([0, 4]))

In [17]:
generate_reduced_quadratic_program_with_qpfs(Q_mi, f_mi, num_active_features, 
                                             num_frozen_features=num_frozen_features, 
                                             alpha = alpha_mi,                                             
                                             frozen_vector_strategy="QPFS")

     pcost       dcost       gap    pres   dres
 0: -2.0878e-01 -1.2429e+00  1e+00  2e-16  3e+00
 1: -2.1131e-01 -2.4517e-01  3e-02  4e-17  1e-01
 2: -2.1695e-01 -2.2031e-01  3e-03  3e-17  5e-03
 3: -2.1807e-01 -2.1829e-01  2e-04  7e-17  1e-16
 4: -2.1814e-01 -2.1815e-01  1e-05  9e-17  8e-17
 5: -2.1814e-01 -2.1814e-01  2e-07  5e-17  1e-16
 6: -2.1814e-01 -2.1814e-01  2e-09  7e-17  9e-17
Optimal solution found.


(array([[1.28814375, 0.85903127, 0.21942317],
        [0.85903127, 1.03095215, 0.23710188],
        [0.21942317, 0.23710188, 1.2939799 ]]),
 array([-0.32384566, -0.32298338, -0.33524141]),
 -0.13828705084525483,
 array([5, 6]),
 array([1, 3, 2]),
 array([0, 4]))

In [18]:
generate_reduced_quadratic_program_with_qpfs(Q_mi, f_mi, num_active_features, 
                                             num_frozen_features=num_frozen_features, 
                                             alpha = alpha_mi,                                             
                                             frozen_vector_strategy="hybrid")

     pcost       dcost       gap    pres   dres
 0: -2.0878e-01 -1.2429e+00  1e+00  2e-16  3e+00
 1: -2.1131e-01 -2.4517e-01  3e-02  4e-17  1e-01
 2: -2.1695e-01 -2.2031e-01  3e-03  3e-17  5e-03
 3: -2.1807e-01 -2.1829e-01  2e-04  7e-17  1e-16
 4: -2.1814e-01 -2.1815e-01  1e-05  9e-17  8e-17
 5: -2.1814e-01 -2.1814e-01  2e-07  5e-17  1e-16
 6: -2.1814e-01 -2.1814e-01  2e-09  7e-17  9e-17
Optimal solution found.


(array([[1.28814375, 0.85903127, 0.21942317],
        [0.85903127, 1.03095215, 0.23710188],
        [0.21942317, 0.23710188, 1.2939799 ]]),
 array([-0.32390225, -0.32324502, -0.33555275]),
 -0.13814563024677667,
 array([5, 6]),
 array([1, 3, 2]),
 array([0, 4]))

In [20]:
# from feature_selection_test import *
# test_construct_mutual_info_relevance_vector()
# test_construct_pearson_corr_relevance_vector()
# test_construct_mutual_info_redundancy_matrix()
# test_construct_pearson_corr_redundancy_matrix()
# test_quadratic_programming_feature_selection()
# test_greedy_mrmr_feature_selection()